# Исследование игрового мобильного приложения

Вы работаете в компании, которая разрабатывает мобильные игры. К вам пришел менеджер с рядом задач по исследованию нескольких аспектов мобильного приложения.

**Задачи:**
1. В первую очередь, его интересует показатель retention. Напишите функцию для его подсчета.
2. Помимо этого, в компании провели A/B тестирование наборов акционных предложений. На основе имеющихся данных определите, какой набор можно считать лучшим и на основе каких метрик стоит принять правильное решение.  
3. Предложите метрики для оценки результатов последнего прошедшего тематического события в игре.  

**Предоставленные данные:**  
* `problem1-reg_data.csv` – данные о времени регистрации
* `problem1-auth_data.csv` – данные о времени захода пользователей в игру  
* `problem2.csv` – результаты A/B теста

## Этапы выполнения проекта

1. [Показатель Retention](#Показатель-Retention)
2. [A/B тестирование](#A/B-тестирование)
3. [Набор метрик для оценки результатов](#Набор-метрик-для-оценки-результатов)

In [33]:
# импорт библиотек
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

%matlpotlib inline

UsageError: Line magic function `%matlpotlib` not found.


## Показатель Retention

### Задача

**Retention** – один из самых важных показателей в компании. Ваша задача – написать функцию, которая будет считать retention игроков (по дням от даты регистрации игрока). 

### Загрузка данных

Изучим данные, предоставленные для проекта.

#### Данные о времени регистрации

Прочитаем файл `problem1-reg_data.csv` и сохраним его в переменной reg.

In [27]:
# чтение файла с данными с сохранением в reg
reg = pd.read_csv('problem1-reg_data.csv', sep=';')

Получение первых 10 строк таблицы.

In [5]:
# получение первых 10 строк таблицы.
reg.head(10)

,reg_ts,uid
0,911382223,1
1,932683089,2
2,947802447,3
3,959523541,4
4,969103313,5
5,977206495,6
6,984222671,7
7,990407778,8
8,995943765,9
9,1000951674,10


Общая информация о данных таблицы reg.

In [8]:
# получение общей информации о данных в таблице df
reg.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Data columns (total 2 columns):
 #   Column  Non-Null Count    Dtype
---  ------  --------------    -----
 0   reg_ts  1000000 non-null  int64
 1   uid     1000000 non-null  int64
dtypes: int64(2)
memory usage: 15.3 MB


Рассмотрим полученную информацию подробнее.

Всего в таблице 2 столбца и 1000000 строк. Присутствует один тип данных: int64.

Подробно разберём, какие столбцы в reg содержат какую информацию:
* reg_ts — время регистриации в формате Unix timestamp
* uid — unique identifier (id пользователя)

Количество значений в столбцах одинаковое, пропущенные значения отсутствуют.

#### Данные о времени захода пользователей в игру

Прочитаем файл `problem1-auth_data.csv` и сохраним его в переменной reg.

In [9]:
# чтение файла с данными с сохранением в auth
auth = pd.read_csv('problem1-auth_data.csv', sep=';')

Получение первых 10 строк таблицы.

In [10]:
# получение первых 10 строк таблицы.
auth.head(10)

,auth_ts,uid
0,911382223,1
1,932683089,2
2,932921206,2
3,933393015,2
4,933875379,2
5,934372615,2
6,934662633,2
7,935002586,2
8,935141232,2
9,935682752,2


Общая информация о данных таблицы reg.

In [11]:
# получение общей информации о данных в таблице df
auth.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9601013 entries, 0 to 9601012
Data columns (total 2 columns):
 #   Column   Dtype
---  ------   -----
 0   auth_ts  int64
 1   uid      int64
dtypes: int64(2)
memory usage: 146.5 MB


In [15]:
# проверка наличия пропущенных значений.
auth.isna().sum()

auth_ts    0
uid        0
dtype: int64

In [19]:
# получим количество строк в таблице
auth.shape[0]

9601013

Рассмотрим полученную информацию подробнее.

Всего в таблице 2 столбца и 9601013 строк. Присутствует один тип данных: int64.

Подробно разберём, какие столбцы в auth содержат какую информацию:
* auth_ts — время захода пользователя в игру в формате Unix timestamp
* uid — unique identifier (id пользователя)

Количество значений в столбцах одинаковое, пропущенные значения отсутствуют.

#### Выводы

Каждая строка таблицы содержит id пользователя и временную отметку о его действии (регистрации или времени захода в игру). Пропуски в данных отсутствуют.

На следующем этапе необходимо будет:
* преобразовать временные отметки из формата Unix timestamp,
* сконвертировать uid в строковый формат (исключить возможные потери нулей и дальнейшие возможные проблемы, правда за счет некоторой незначительной для данного объема данных потери эффективности)  
* исследовать данные на наличие аномалий (например, регистрации с повторным присвоением уже использованного uid, аномальную частоту заходов пользователей в игру (отсечь ботов) и т.д.)